In [10]:
import requests
import pandas as pd
import boto3

In [12]:
API_KEY = "6c13523c85c4fae3bb0d6d5ec7a89784"  
BUCKET_NAME = "soccer-data-bucket-nav"  
CSV_FILE_NAME = "worldcup_odds.csv"
S3_KEY = f"data/{CSV_FILE_NAME}"

# ---------------------------------------
# STEP 2: Fetch World Cup odds
# ---------------------------------------
url = "https://api.the-odds-api.com/v4/sports/soccer_fifa_world_cup_winner/odds/"
params = {
    "regions": "us",          # you can also try "eu" or "uk"
    "markets": "outrights",
    "oddsFormat": "decimal",
    "apiKey": API_KEY
}

response = requests.get(url, params=params)
response.raise_for_status()
data = response.json()

# ---------------------------------------
# STEP 3: Parse odds data
# ---------------------------------------
rows = []

for event in data:
    for bookmaker in event.get("bookmakers", []):
        for market in bookmaker.get("markets", []):
            if market["key"] == "outrights":
                for outcome in market.get("outcomes", []):
                    rows.append({
                        "Team": outcome["name"],
                        "Odds": outcome["price"],
                        "Bookmaker": bookmaker["title"],
                        "Last Updated": bookmaker["last_update"]
                    })

# Convert to DataFrame
df = pd.DataFrame(rows)
df = df.sort_values(by="Odds", ascending=True).drop_duplicates(subset=["Team"])
df.reset_index(drop=True, inplace=True)

# ---------------------------------------
# STEP 4: Save CSV locally
# ---------------------------------------
df.to_csv(CSV_FILE_NAME, index=False)
print(f"✅ CSV saved: {CSV_FILE_NAME}")

# ---------------------------------------
# STEP 5: Upload to S3
# ---------------------------------------
s3 = boto3.client("s3")
s3.upload_file(CSV_FILE_NAME, BUCKET_NAME, S3_KEY)
print(f"✅ File uploaded to s3://{BUCKET_NAME}/{S3_KEY}")

✅ CSV saved: worldcup_odds.csv
✅ File uploaded to s3://soccer-data-bucket-nav/data/worldcup_odds.csv


In [16]:
df.head()

,Team,Odds,Bookmaker,Last Updated
0,Spain,6.0,DraftKings,2025-06-08T20:36:03Z
1,France,6.5,FanDuel,2025-06-08T20:17:02Z
2,Brazil,7.0,FanDuel,2025-06-08T20:17:02Z
3,England,7.5,Bovada,2025-06-08T20:37:41Z
4,Argentina,9.0,BetMGM,2025-06-08T20:39:46Z
